# Interactive Visualization

This notebook creates interactive visualizations of the clustering results using:
- UMAP and t-SNE for dimensionality reduction
- Plotly for interactive plots
- 2D and 3D visualizations
- Cluster characteristic analysis

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from suricata_rule_clustering import viz

# Set display options
pd.set_option('display.max_columns', None)

## 1. Load Data

In [ ]:
# Load feature matrix and clustered DataFrame
X = np.load('../data/feature_matrix.npy')
df = pd.read_pickle('../data/clustered_rules.pkl')
labels = np.load('../data/cluster_labels.npy')

print(f"Feature matrix shape: {X.shape}")
print(f"DataFrame shape: {df.shape}")
print(f"Number of clusters: {len(set(labels))}")

## 2. UMAP 2D Visualization

In [ ]:
# Reduce dimensions using UMAP to 2D
X_umap_2d = viz.reduce_dimensions(
    X,
    method='umap',
    n_components=2,
    n_neighbors=15,
    min_dist=0.1
)

print(f"UMAP 2D shape: {X_umap_2d.shape}")

In [ ]:
# Create interactive 2D plot
fig_umap_2d = viz.plot_clusters_2d(
    X_umap_2d,
    labels,
    df=df,
    title="Suricata Rule Clusters - UMAP 2D",
    hover_data=['msg', 'classtype', 'priority', 'action', 'protocol']
)

fig_umap_2d.show()

In [ ]:
# Save the figure
viz.save_figure(fig_umap_2d, 'umap_2d_clusters', '../outputs')

## 3. UMAP 3D Visualization

In [ ]:
# Reduce dimensions using UMAP to 3D
X_umap_3d = viz.reduce_dimensions(
    X,
    method='umap',
    n_components=3,
    n_neighbors=15,
    min_dist=0.1
)

print(f"UMAP 3D shape: {X_umap_3d.shape}")

In [ ]:
# Create interactive 3D plot
fig_umap_3d = viz.plot_clusters_3d(
    X_umap_3d,
    labels,
    df=df,
    title="Suricata Rule Clusters - UMAP 3D",
    hover_data=['msg', 'classtype', 'priority']
)

fig_umap_3d.show()

In [ ]:
# Save the figure
viz.save_figure(fig_umap_3d, 'umap_3d_clusters', '../outputs')

## 4. t-SNE 2D Visualization

t-SNE is another popular dimensionality reduction technique. It may take longer than UMAP.

In [ ]:
# Reduce dimensions using t-SNE to 2D
X_tsne_2d = viz.reduce_dimensions(
    X,
    method='tsne',
    n_components=2,
    perplexity=30
)

print(f"t-SNE 2D shape: {X_tsne_2d.shape}")

In [ ]:
# Create interactive 2D plot
fig_tsne_2d = viz.plot_clusters_2d(
    X_tsne_2d,
    labels,
    df=df,
    title="Suricata Rule Clusters - t-SNE 2D",
    hover_data=['msg', 'classtype', 'priority', 'action', 'protocol']
)

fig_tsne_2d.show()

In [ ]:
# Save the figure
viz.save_figure(fig_tsne_2d, 'tsne_2d_clusters', '../outputs')

## 5. Cluster Analysis Visualizations

In [ ]:
# Plot cluster sizes
fig_sizes = viz.plot_cluster_sizes(labels, "Cluster Sizes")
fig_sizes.show()
viz.save_figure(fig_sizes, 'cluster_sizes', '../outputs')

In [ ]:
# Plot priority distribution by cluster
if 'priority' in df.columns:
    fig_priority = viz.plot_cluster_characteristics(
        df,
        labels,
        feature='priority',
        plot_type='box'
    )
    fig_priority.show()
    viz.save_figure(fig_priority, 'priority_by_cluster', '../outputs')

In [ ]:
# Plot classtype distribution by cluster
if 'classtype' in df.columns:
    fig_classtype = viz.plot_classtype_distribution(
        df,
        labels,
        top_n=15
    )
    fig_classtype.show()
    viz.save_figure(fig_classtype, 'classtype_distribution', '../outputs')

In [ ]:
# Plot message length distribution by cluster
if 'msg_length' in df.columns:
    fig_msg_length = viz.plot_cluster_characteristics(
        df,
        labels,
        feature='msg_length',
        plot_type='violin'
    )
    fig_msg_length.show()
    viz.save_figure(fig_msg_length, 'msg_length_by_cluster', '../outputs')

## 6. Comprehensive Dashboard

Create a dashboard with multiple visualizations.

In [ ]:
# Prepare metrics
from suricata_rule_clustering import clustering

metrics = clustering.evaluate_clustering(X, labels)

# Create dashboard
fig_dashboard = viz.create_cluster_dashboard(
    X,
    X_umap_2d,
    labels,
    df,
    metrics
)

fig_dashboard.show()

In [ ]:
# Save dashboard
viz.save_figure(fig_dashboard, 'clustering_dashboard', '../outputs')

## 7. Export Cluster Examples

Export representative rules from each cluster for manual inspection.

In [ ]:
# Get representative rules from each cluster
msg_col = 'msg' if 'msg' in df.columns else 'message'

cluster_examples = []

for cluster_id in sorted(df['cluster'].unique()):
    cluster_rules = df[df['cluster'] == cluster_id]
    
    # Get top 5 rules from this cluster
    examples = cluster_rules.head(5)[
        ['cluster', msg_col, 'classtype', 'priority', 'action', 'protocol', 'raw_rule']
    ]
    
    cluster_examples.append(examples)

# Combine and save
examples_df = pd.concat(cluster_examples, ignore_index=True)
examples_df.to_csv('../outputs/cluster_examples.csv', index=False)

print(f"Exported {len(examples_df)} example rules to cluster_examples.csv")
examples_df.head(20)

## 8. Save Reduced Dimensions

Save the reduced dimension representations for future use.

In [ ]:
# Save reduced dimensions
np.save('../data/umap_2d.npy', X_umap_2d)
np.save('../data/umap_3d.npy', X_umap_3d)
np.save('../data/tsne_2d.npy', X_tsne_2d)

print("Reduced dimensions saved")

## Summary

We have successfully created interactive visualizations including:
- 2D and 3D UMAP projections
- 2D t-SNE projection
- Cluster size distributions
- Cluster characteristic analysis (priority, classtype, message length)
- Comprehensive dashboard

All visualizations have been saved to the `outputs/` directory as interactive HTML files that can be opened in any web browser.

## Next Steps

- Examine the cluster examples in `outputs/cluster_examples.csv`
- Open the HTML visualizations in a browser for interactive exploration
- Fine-tune clustering parameters based on the visual results
- Identify patterns and relationships between similar rules